# MDG NAP Preliminary Analysis

In [48]:
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import plotly.plotly as py
import plotly.graph_objs as go

engine = create_engine('postgresql://postgres@localhost:5433/cartodb_user_fddc7a0f-c829-4ec0-a4b8-212ec7ce9ad1_db')
sql_rast_mean_by_region = """
SELECT {group_by},
(ST_SummaryStatsAgg(ST_Clip(the_raster_webmercator, the_geom_webmercator, {na_value}, true), 1, true)).* 
FROM {raster}, {poly}
WHERE ST_Intersects(the_raster_webmercator, the_geom_webmercator)
GROUP BY {group_by};
"""

# General function to compare series
def plot_comparison(df, x_var, y_var, x_label, y_label, size=1):
    trace = go.Scatter(
        x = df[x_var],
        y = df[y_var],
        text = df.index,
        mode = 'markers',
        marker = dict (
            size = size
        )
    )
    layout = go.Layout(
        xaxis=dict(title=x_label),
        yaxis=dict(title=y_label)
    )
    data = [trace]
    fig = go.Figure(data=data, layout=layout)
    return(py.iplot(fig, filename='basic-scatter'))

# General function to compare two rasters by adm2 region
def get_rast_mean_by_region(rast, region="(select * from gadm28_adm2 where iso in ('MDG')) as regions"):
    key = {'raster': rast[1], # This is the raster table name
           'poly': region,
           'na_value': -9999,
           'group_by': 'name_2'}
    df = pd.read_sql_query(sql_rast_mean_by_region.format(**key), con=engine)
    df['variable'] = rast[0] # This is a short name for the raster
    return(df)

In [50]:
plot_comparison(df_commune,
                x_var='food_insecurity', y_var='precip',
                x_label='Percentage food insecure in ??', y_label='Precipitation change (%)',
                size='pop')

                           precip       temp  food_insecurity
first_nom_                                                   
NaN                      0.073770   2.140697              0.0
Bekobay                  2.122728  11.261433             30.0
Ambohimiera             -6.410193  14.191587             90.0
Antanimandry            -4.089653  21.092957             20.0
Manambolo               -1.956413  12.906124             65.0
Ampasimaneva            -5.966038  15.001510             80.0
Andapa                  -1.757359  12.985084             10.0
Andaingo                -3.669198  14.901495             35.0
Anjeva                  -4.302230  16.623657             30.0
Mandronarivo            -2.032313  12.818932             37.0
Ankalalobe               0.113270  11.970594             95.0
Maromavo                 1.182548  11.321694             30.0
Besalampy                2.046519  10.865229             26.0
Manakana                -3.049476  14.142923             77.0
Manakana

## Agriculture / water

### Climate trends

In [3]:
rast_comp =({'precip':'total_annual_rainfall_multimodel_pctdiff_mean_85_50_mdg',
             'temp':'total_annual_temperature_multimodel_pctdiff_mean_8570_mdg'})
df = pd.concat([get_rast_mean_by_region(rast) for rast in rast_comp.items()])
plot_comparison(df.pivot(index='name_2', columns='variable', values='mean'),
                x_var='temp', y_var='precip',
                x_label='Temperature change (°C)', y_label='Precipitation change (%)')

### Health versus climate

In [16]:
# Pull precipitation trends
key = {'raster': 'total_annual_rainfall_multimodel_pctdiff_mean_85_50_mdg',
       'poly': 'sdr_subnational_data_dhs_2008',
       'na_value': -9999,
       'group_by': 'dhsregen'}
df_rain = pd.read_sql_query(sql_rast_mean_by_region.format(**key), con=engine)
df_rain = df_rain.rename(columns={'mean':'precip'})

# Pull temperature trends
key = {'raster': 'total_annual_temperature_multimodel_pctdiff_mean_8570_mdg',
       'poly': 'sdr_subnational_data_dhs_2008',
       'na_value': -9999,
       'group_by': 'dhsregen'}
df_temp = pd.read_sql_query(sql_rast_mean_by_region.format(**key), con=engine)
df_temp = df_temp.rename(columns={'mean':'temp'})

# Merge both climate datasets
df_clim = pd.merge(df_rain[['dhsregen', 'precip']], df_temp[['dhsregen', 'temp']])

# Pull health data
df_health = pd.read_sql_query('select cnnutscha2 as stunting, cmecmrcimr as imr, mlitnapacc as pct_access_nets, dhsregen from sdr_subnational_data_dhs_2008', con=engine)
df_health = pd.merge(df_clim[['dhsregen', 'precip', 'temp']], df_health)
df_health = df_health.set_index('dhsregen')

#### Stunting

In [17]:
plot_comparison(df_health,
                x_var='stunting', y_var='precip',
                x_label='Children that are stunted (%)', y_label='Precipitation change (%)')

In [18]:
plot_comparison(df_health,
                x_var='stunting', y_var='temp',
                x_label='Children that are stunted (%)', y_label='Temperature change (%)')

#### Infant mortality rates

In [19]:
plot_comparison(df_health,
                x_var='imr', y_var='precip',
                x_label='Infant mortality rate', y_label='Precipitation change (%)')

In [20]:
plot_comparison(df_health,
                x_var='imr', y_var='temp',
                x_label='Infant mortality rate', y_label='Temperature change (%)')

#### Access to bed nets

In [21]:
plot_comparison(df_health,
                x_var='pct_access_nets', y_var='precip',
                x_label='Percentage with access to insecticide treated nets', y_label='Precipitation change (%)')

In [22]:
plot_comparison(df_health,
                x_var='pct_access_nets', y_var='temp',
                x_label='Percentage with access to insecticide treated nets', y_label='Temperature change (%)')

#### Climate food security

In [45]:
# Pull precipitation trends
key = {'raster': 'total_annual_rainfall_multimodel_pctdiff_mean_85_50_mdg',
       'poly': 'communes_poverty_fid_2007',
       'na_value': -9999,
       'group_by': 'first_nom_'}
df_rain = pd.read_sql_query(sql_rast_mean_by_region.format(**key), con=engine)
df_rain = df_rain.rename(columns={'mean':'precip'})

# Pull temperature trends
key = {'raster': 'total_annual_temperature_multimodel_pctdiff_mean_8570_mdg',
       'poly': 'communes_poverty_fid_2007',
       'na_value': -9999,
       'group_by': 'first_nom_'}
df_temp = pd.read_sql_query(sql_rast_mean_by_region.format(**key), con=engine)
df_temp = df_temp.rename(columns={'mean':'temp'})

# Pull population
key = {'raster': 'lspop2014',
       'poly': 'communes_poverty_fid_2007',
       'na_value': -9999,
       'group_by': 'first_nom_'}
df_pop = pd.read_sql_query(sql_rast_mean_by_region.format(**key), con=engine)
df_pop = df_pop.rename(columns={'sum':'pop'})
#print df_pop[df_pop['pop'] == df_pop['pop'].max()]
#print df_pop['pop'].sum()

# Merge climate and population datasets
df_clim = pd.merge(df_rain[['first_nom_', 'precip']], df_temp[['first_nom_', 'temp']])
df_clim = pd.merge(df_clim, df_pop[['first_nom_', 'pop']])

# Pull health data
df_commune = pd.read_sql_query('select poor_per as food_insecurity, first_nom_ from communes_poverty_fid_2007', con=engine)
df_commune = pd.merge(df_clim[['first_nom_', 'precip', 'temp']], df_commune)
df_commune = df_commune.set_index('first_nom_')

In [49]:
plot_comparison(df_commune,
                x_var='food_insecurity', y_var='precip',
                x_label='Percentage food insecure in ??', y_label='Precipitation change (%)',
                size='pop')

In [32]:
plot_comparison(df_commune,
                x_var='food_insecurity', y_var='temp',
                x_label='Percentage food insecure in ??', y_label='Temperature change (%)')

                           precip       temp  food_insecurity
first_nom_                                                   
NaN                      0.073770   2.140697              0.0
Bekobay                  2.122728  11.261433             30.0
Ambohimiera             -6.410193  14.191587             90.0
Antanimandry            -4.089653  21.092957             20.0
Manambolo               -1.956413  12.906124             65.0
Ampasimaneva            -5.966038  15.001510             80.0
Andapa                  -1.757359  12.985084             10.0
Andaingo                -3.669198  14.901495             35.0
Anjeva                  -4.302230  16.623657             30.0
Mandronarivo            -2.032313  12.818932             37.0
Ankalalobe               0.113270  11.970594             95.0
Maromavo                 1.182548  11.321694             30.0
Besalampy                2.046519  10.865229             26.0
Manakana                -3.049476  14.142923             77.0
Manakana

#### Climate versus essential natural capital for food security

#### Climate versus rice